In [35]:
#Set environment variables for your notebook
import os 
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/kellywisneski/Downloads/msds434-final-5dd240442926.json'

In [9]:
#!pip install google-cloud-automl --user

In [31]:
#Imports google cloud client library and initiates BQ service
from google.cloud import bigquery
from google.cloud import automl_v1beta1 as automl
from google.api_core import exceptions
#from google.cloud import storage

In [32]:
#REPLACE THIS WITH YOUR OWN GOOGLE PROJECT ID
PROJECT_ID = 'msds434-final'
#REPLACE THIS WITH A NEW BUCKET NAME. NOTE: BUCKET NAMES MUST BE GLOBALLY UNIQUE
BUCKET_NAME = 'msds434_final_bucket_v1'
#Note: the bucket_region must be us-central1.
REGION = 'us-central1'

In [33]:
bigquery_client = bigquery.Client()
#storage_client = storage.Client(project=PROJECT_ID)
#tables_gcs_client = automl.GcsClient(client=storage_client, bucket_name=BUCKET_NAME)
#automl_client = automl.AutoMlClient()
# Note: AutoML Tables currently is only eligible for region us-central1. 
#prediction_client = automl.PredictionServiceClient()
# Note: This line runs unsuccessfully without each one of these parameters
#tables_client = automl.TablesClient(project=PROJECT_ID, region=BUCKET_REGION, client=automl_client, gcs_client=tables_gcs_client, prediction_client=prediction_client)
client = automl.TablesClient(project=PROJECT_ID, region=REGION)

In [13]:
#Write Query on BQ
query = """
SELECT
    trip_start_timestamp,
    trip_end_timestamp,
    trip_seconds,
    trip_miles,
    pickup_census_tract,
    dropoff_census_tract,
    pickup_community_area,
    dropoff_community_area,
    fare,
    tolls,
    extras,
    trip_total,
    payment_type,
    company,
    pickup_longitude,
    pickup_latitude,
    dropoff_longitude,
    dropoff_latitude,
    IF((tips/fare >= 0.2),
      1,
      0) AS tip_bin
FROM
    `bigquery-public-data.chicago_taxi_trips.taxi_trips`
WHERE
    trip_miles > 0
    AND fare > 0
LIMIT
  10000000
  """

In [33]:
#Run the query and write result to a pandas data frame
query_results = bigquery_client.query(query)
df = query_results.to_dataframe()

In [34]:
#View top few rows of result
df.head()

,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,fare,tolls,extras,trip_total,payment_type,company,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,tip_bin
0,2017-11-19 11:00:00+00:00,2017-11-19 11:15:00+00:00,720,2.3,17031081000,17031281900,8,28,9.75,0.0,0.0,12.25,Credit Card,Globe Taxi,-87.632109,41.900266,-87.642649,41.879255,1
1,2016-06-13 18:00:00+00:00,2016-06-13 18:15:00+00:00,1200,3.6,17031070200,17031281900,7,28,13.75,0.0,1.0,14.75,Cash,City Service,-87.646293,41.929078,-87.642649,41.879255,0
2,2017-11-22 15:45:00+00:00,2017-11-22 15:45:00+00:00,542,1.4,17031838100,17031281900,28,28,7.50,0.0,1.0,8.50,Cash,Taxi Affiliation Service Yellow,-87.671654,41.878667,-87.642649,41.879255,0
3,2017-11-14 16:00:00+00:00,2017-11-14 16:00:00+00:00,230,0.6,17031031400,17031060900,3,6,4.75,0.0,0.0,4.75,Cash,Chicago Carriage Cab Corp,-87.645379,41.962179,-87.643965,41.949829,0
4,2016-06-17 17:00:00+00:00,2016-06-17 17:15:00+00:00,900,1.4,17031060400,17031060900,6,6,9.00,0.0,0.0,9.00,Cash,City Service,-87.666536,41.950673,-87.643965,41.949829,0


In [35]:
#quick summary stats
df.describe()

,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,fare,tolls,extras,trip_total,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,tip_bin
count,9.999463e+06,1.000000e+07,6.990160e+06,6.835367e+06,9.574590e+06,8.991285e+06,1.000000e+07,8.751432e+06,1.000000e+07,1.000000e+07,9.574845e+06,9.574845e+06,8.991904e+06,8.991904e+06,1.000000e+07
mean,8.996159e+02,4.877747e+00,1.703138e+10,1.703127e+10,2.615229e+01,2.091420e+01,1.445825e+01,2.180666e-03,1.349599e+00,1.757994e+01,-8.766242e+01,4.189717e+01,-8.764255e+01,4.188987e+01,3.161479e-01
std,1.332485e+03,1.257470e+01,3.373217e+05,2.597870e+05,2.073605e+01,1.596370e+01,3.481857e+01,1.154484e+00,5.835628e+00,3.720892e+01,8.131848e-02,4.160661e-02,5.042823e-02,3.880499e-02,4.649714e-01
min,0.000000e+00,1.000000e-02,1.703101e+10,1.703101e+10,1.000000e+00,1.000000e+00,1.000000e-02,0.000000e+00,0.000000e+00,1.000000e-02,-8.791362e+01,4.166014e+01,-8.791362e+01,4.166014e+01,0.000000e+00
25%,3.910000e+02,1.000000e+00,1.703108e+10,1.703108e+10,8.000000e+00,8.000000e+00,6.250000e+00,0.000000e+00,0.000000e+00,7.500000e+00,-8.765177e+01,4.188099e+01,-8.764265e+01,4.187926e+01,0.000000e+00
50%,6.130000e+02,1.800000e+00,1.703128e+10,1.703124e+10,2.800000e+01,1.400000e+01,8.750000e+00,0.000000e+00,0.000000e+00,1.025000e+01,-8.763275e+01,4.189197e+01,-8.762621e+01,4.189092e+01,0.000000e+00
75%,1.080000e+03,5.100000e+00,1.703184e+10,1.703132e+10,3.200000e+01,3.200000e+01,1.525000e+01,0.000000e+00,1.000000e+00,1.750000e+01,-8.762197e+01,4.190007e+01,-8.761971e+01,4.189833e+01,1.000000e+00
max,8.639600e+04,3.460000e+03,1.703198e+10,1.703198e+10,7.700000e+01,7.700000e+01,9.898980e+03,3.333330e+03,9.243000e+03,9.997960e+03,-8.753139e+01,4.202122e+01,-8.753139e+01,4.202122e+01,1.000000e+00


In [45]:
#create dataset
dataset = bigquery_client.create_dataset("dataset_v1")

In [38]:
#define target column
target_column = 'tip_bin'

In [46]:
#define model query

model_query = f"""
CREATE OR REPLACE MODEL `dataset_v1.tip-model-v1`
OPTIONS(MODEL_TYPE = 'automl_classifier', 
BUDGET_HOURS = 1.0, 
OPTIMIZATION_OBJECTIVE = 'MINIMIZE_LOG_LOSS',
INPUT_LABEL_COLS=['{target_column}'])
AS 
{query}
"""

In [47]:
#create model

model = bigquery_client.query(model_query)
model

QueryJob<project=msds434-final, location=US, id=d25d6854-68e7-4b0d-8c1c-9f70b74ad5ef>

In [48]:
results = model.result()
results

In [43]:
#get training info
model_training_info_query = """
SELECT
  *
FROM
  ML.TRAINING_INFO(MODEL `dataset_v1.tip-model-v1`)
"""

model_training_info = bigquery_client.query(model_training_info_query).to_dataframe()

In [44]:
#see training info
model_training_info

,training_run,iteration,loss,eval_loss,learning_rate,duration_ms
0,0,0,0.0,0.032088,NaN,5770800


In [45]:
#evaluate
model_eval_query = """
SELECT
  *
FROM
  ML.EVALUATE(MODEL `dataset_v1.tip-model-v1`)
"""

model_eval = bigquery_client.query(model_eval_query).to_dataframe()

In [46]:
#see eval info
model_eval

,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.988138,0.996572,0.995216,0.992337,0.032088,0.998212


In [ ]:
#train new model based on updated feature importance and new dataset split
#trip total, fare, extras, payment_type

In [51]:
#create dataset
dataset = bigquery_client.create_dataset("dataset_v2")

In [61]:
#create view with split data

#predict
v2_view_query = """
CREATE OR REPLACE VIEW
  `dataset_v2.input_view` AS
SELECT
    fare,
    extras,
    trip_total,
    payment_type,
    IF((tips/fare >= 0.2),
      1,
      0) AS tip_bin,
  CASE
    WHEN MOD(trip_seconds, 10) < 8 THEN 'training'
    WHEN MOD(trip_seconds, 10) = 8 THEN 'evaluation'
    WHEN MOD(trip_seconds, 10) = 9 THEN 'prediction'
  END AS dataframe
FROM
    `bigquery-public-data.chicago_taxi_trips.taxi_trips`
WHERE
    trip_miles > 0
    AND fare > 0
LIMIT
  10000000
"""

v2_view = bigquery_client.query(v2_view_query).to_dataframe()

In [63]:
#define target column
target_column = 'tip_bin'

In [74]:
#define model query

v2_model_query = f"""
CREATE OR REPLACE MODEL `dataset_v2.tip-model-v2`
OPTIONS(model_type='LOGISTIC_REG',
    auto_class_weights=TRUE,
    data_split_method='NO_SPLIT',
    input_label_cols=['{target_column}'],
    max_iterations=15)
AS 
SELECT
  * EXCEPT(dataframe)
FROM
  `dataset_v2.input_view`
WHERE
  dataframe = 'training'
"""

In [75]:
#create model

v2_model = bigquery_client.query(v2_model_query)
v2_model

QueryJob<project=msds434-final, location=US, id=c3162626-595d-462d-a0ad-b1a42fd7ad25>

In [76]:
#evaluate query
v2_model_eval_query = """
SELECT
  *
FROM
  ML.EVALUATE (MODEL `dataset_v2.tip-model-v2`,
    (
    SELECT
      *
    FROM
      `dataset_v2.input_view`
    WHERE
      dataframe = 'evaluation'
    )
  )
"""

v2_model_eval = bigquery_client.query(v2_model_eval_query).to_dataframe()

In [77]:
#print eval
v2_model_eval

,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.976327,0.971712,0.985788,0.974014,0.039388,0.999781


In [78]:
#predict
v2_model_pred_query = """
SELECT
  *
FROM
  ML.PREDICT (MODEL `dataset_v2.tip-model-v2`,
    (
    SELECT
      *
    FROM
      `dataset_v2.input_view`
    WHERE
      dataframe = 'prediction'
     )
  )
"""

v2_model_pred = bigquery_client.query(v2_model_pred_query).to_dataframe()

In [79]:
#print prediction
v2_model_pred

,predicted_tip_bin,predicted_tip_bin_probs,fare,extras,trip_total,payment_type,tip_bin,dataframe
0,1,"[{'label': 0, 'prob': 0.03872327879071236}, {'...",36.50,25.5,71.88,Credit Card,1,prediction
1,0,"[{'label': 0, 'prob': 0.9999975562095642}, {'l...",14.00,0.0,14.00,Cash,0,prediction
2,0,"[{'label': 0, 'prob': 0.9999949336051941}, {'l...",12.60,0.0,12.60,Cash,0,prediction
3,1,"[{'label': 0, 'prob': 0.0007965390104800463}, ...",0.01,0.0,5.01,Credit Card,1,prediction
4,0,"[{'label': 0, 'prob': 0.9999361038208008}, {'l...",28.80,0.0,28.80,Cash,0,prediction
...,...,...,...,...,...,...,...,...
618246,0,"[{'label': 0, 'prob': 0.9999897480010986}, {'l...",7.45,0.0,7.45,Cash,0,prediction
618247,0,"[{'label': 0, 'prob': 0.9999956488609314}, {'l...",6.45,1.0,7.45,Cash,0,prediction
618248,0,"[{'label': 0, 'prob': 0.9999897480010986}, {'l...",7.45,0.0,7.45,Cash,0,prediction
618249,0,"[{'label': 0, 'prob': 0.9427914023399353}, {'l...",6.25,0.0,7.45,Credit Card,0,prediction


In [80]:
#predict explanations
v2_model_pred__expl_query = """
SELECT
  *
FROM
  ML.EXPLAIN_PREDICT(MODEL `dataset_v2.tip-model-v2`,
    (
    SELECT
      *
    FROM
      `dataset_v2.input_view`
    WHERE
      dataframe = 'evaluation'),
    STRUCT(3 as top_k_features))
"""

v2_model_expl_pred = bigquery_client.query(v2_model_pred__expl_query).to_dataframe()

In [81]:
#print prediction explanations
v2_model_expl_pred

,predicted_tip_bin,probability,top_feature_attributions,baseline_prediction_value,prediction_value,approximation_error,fare,extras,trip_total,payment_type,tip_bin,dataframe
0,0,0.845760,"[{'feature': 'payment_type', 'attribution': -1...",-0.250858,-1.701729,0.0,17.25,0.0,17.25,Cash,0,evaluation
1,1,0.807903,"[{'feature': 'payment_type', 'attribution': 1....",-0.250858,1.436444,0.0,14.50,0.0,18.00,Credit Card,1,evaluation
2,0,0.845592,"[{'feature': 'payment_type', 'attribution': -1...",-0.250858,-1.700438,0.0,17.50,0.0,17.50,Cash,0,evaluation
3,1,0.802885,"[{'feature': 'payment_type', 'attribution': 1....",-0.250858,1.404424,0.0,8.75,0.0,11.75,Credit Card,1,evaluation
4,1,0.835939,"[{'feature': 'payment_type', 'attribution': 1....",-0.250858,1.628314,0.0,46.00,0.0,55.80,Credit Card,1,evaluation
...,...,...,...,...,...,...,...,...,...,...,...,...
797509,0,0.852251,"[{'feature': 'payment_type', 'attribution': -1...",-0.250858,-1.752363,0.0,7.45,0.0,7.45,Cash,0,evaluation
797510,0,0.852251,"[{'feature': 'payment_type', 'attribution': -1...",-0.250858,-1.752363,0.0,7.45,0.0,7.45,Cash,0,evaluation
797511,0,0.850067,"[{'feature': 'payment_type', 'attribution': -1...",-0.250858,-1.735130,0.0,6.45,1.0,7.45,Cash,0,evaluation
797512,0,0.852251,"[{'feature': 'payment_type', 'attribution': -1...",-0.250858,-1.752363,0.0,7.45,0.0,7.45,Cash,0,evaluation


In [84]:
!pip install --upgrade google-api-python-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 4.0 MB/s eta 0:00:0000:0100:01


In [93]:
import googleapiclient.discovery
from google.api_core.client_options import ClientOptions

In [94]:

def predict_json(project, region, model, instances, version=None):
    """Send json data to a deployed model for prediction.

    Args:
        project (str): project where the Cloud ML Engine Model is deployed.
        region (str): regional endpoint to use; set to None for ml.googleapis.com
        model (str): model name.
        instances ([Mapping[str: Any]]): Keys should be the names of Tensors
            your deployed model expects as inputs. Values should be datatypes
            convertible to Tensors, or (potentially nested) lists of datatypes
            convertible to tensors.
        version: str, version of the model to target.
    Returns:
        Mapping[str: any]: dictionary of prediction results defined by the
            model.
    """
    # Create the ML Engine service object.
    # To authenticate set the environment variable
    # GOOGLE_APPLICATION_CREDENTIALS=<path_to_service_account_file>
    prefix = "{}-ml".format(region) if region else "ml"
    api_endpoint = "https://{}.googleapis.com".format(prefix)
    client_options = ClientOptions(api_endpoint=api_endpoint)
    service = googleapiclient.discovery.build(
        'ml', 'v1', client_options=client_options)
    name = 'projects/{}/models/{}'.format(project, model)

    if version is not None:
        name += '/versions/{}'.format(version)

    response = service.projects().predict(
        name=name,
        body={'instances': instances}
    ).execute()

    if 'error' in response:
        raise RuntimeError(response['error'])

    return response['predictions']

In [99]:
project="msds434-final"
model="TIP_MODEL"
version="v1"
region="us-central1"

In [137]:
#instances={"instances": [{"values": [17.75, 1.0, 7.45, "Cash"], "key": 1}]}
#instances=[{'fare': [17.75],
#  'extras': [1.0],
#  'trip_total': [7.45],
#  'payment_type': ["Cash"]}]
#instances={"values": [17.75, 1.0, 7.45, "Cash"], "key": 1}
#instances=[17.75, 1.0, 7.45, "Cash"]
#instances={"instances": [{"fare": 17.75,"extras": 1.0,"trip_total": 7.45,"payment_type": "Cash"}]}
instances=[{"fare": 17.75,"extras": 1.0,"trip_total": 7.45,"payment_type": "Cash"}]

In [138]:
instances

[{'fare': 17.75, 'extras': 1.0, 'trip_total': 7.45, 'payment_type': 'Cash'}]

In [139]:
predict_json(project, region, model, instances, version)

[{'tip_bin_values': ['1', '0'],
  'predicted_tip_bin': ['0'],
  'tip_bin_probs': [0.02135814720051792, 0.9786418527994821]}]